In [21]:
!python -m pip install Faker

In [22]:
from numpy.random import choice
import random
from faker import Faker
from faker.providers import phone_number
import json

In [23]:
fake = Faker()
fake.add_provider(phone_number)
pnumbers = [fake.phone_number() for _ in range(100)]
calls = [ {"caller": random.choice(pnumbers), "recipient": random.choice(pnumbers), "duration_s": random.randint(5, 600)} for _ in range(10**5) ]
suspects = choice(pnumbers, size=10, replace=False)
with open("numbers.txt", "w+") as file:
    file.writelines([f'{str(number)}\n' for number in pnumbers])
with open("suspects.txt", "w+") as file:
    file.writelines([f'{str(number)}\n' for number in suspects])
with open("calls.txt", "w+") as file:
    file.writelines([f'caller:{call.get("caller")}|recipient:{call.get("recipient")}|duration_s:{call.get("duration_s")}\n' for call in calls])

In [24]:
with open('calls.txt', 'r') as f:
    calls = f.readlines()
with open('suspects.txt', 'r') as f:
    suspects = f.readlines()
characters_to_remove = '().-'
with open('new_calls.txt', 'w') as f:
    for call in calls:
        new_call = call
        for c in characters_to_remove:
            new_call = new_call.replace(c, '')
        new_call = new_call.split('|')
        caller = new_call[0]
        caller = caller.split(':')[1]
        caller = caller.split('x')[0]
        caller = caller[-10:]
        int(caller)
        assert len(caller) == 10
        caller = 'caller:' + caller
        recipient = new_call[1]
        recipient = recipient.split(':')[1]
        recipient = recipient.split('x')[0]
        recipient = recipient[-10:]
        int(recipient)
        assert len(recipient) == 10
        recipient = 'recipient:' + recipient
        new_call = caller + '|' + recipient + '|' + new_call[2]
        f.write(new_call)
with open('new_suspects.txt', 'w') as f:
    for sus in suspects:
        new_sus = sus
        for c in characters_to_remove:
            new_sus = new_sus.replace(c, '')
        new_sus = new_sus.split('x')[0]
        new_sus = new_sus.split('\n')[0]
        new_sus = new_sus[-10:]
        int(new_sus)
        assert len(new_sus) == 10
        f.write(new_sus)
        f.write('\n')
with open('new_calls.txt', 'r') as f:
    calls = f.readlines()
with open('new_suspects.txt', 'r') as f:
    suspects = f.readlines()


In [25]:
N = 5

with open('new_suspects.txt', 'w') as f:
    current_suspects = suspects.copy()
    base_suspects = set(suspects)
    all_suspects = set(suspects)
    proofs = dict()
    for _ in range(N):
        new_suspects = set()
        for call in calls:
            caller = call.split('|')[0].split(':')[1]
            recipient = call.split('|')[1].split(':')[1]
            duration_s = call.split('|')[2].split(':')[1]
            if recipient + '\n' in current_suspects and int(duration_s) > 120:
                try:
                    proofs[caller].append({'recipient': recipient, 'duration': int(duration_s)})
                except  KeyError:
                    proofs[caller] = [{'recipient': recipient, 'duration': int(duration_s)}]
                if len(proofs[caller]) > 2:
                    new_suspects.add(caller + '\n')
        f.writelines(new_suspects - all_suspects)
        f.write('kek\n')
        all_suspects = all_suspects | new_suspects
        current_suspects = new_suspects.copy()
    with open('proofs.json', 'w') as j_f:
        j_f.write(json.dumps(proofs))

